In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
from IPython import display
display.clear_output()
from fastai.tabular import *

In [0]:
# import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
dep_var = ['Y']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

df_D = pd.get_dummies(df[cols])
test_df_D = pd.get_dummies(test_df[cols])

In [3]:
df_D=df_D.fillna(0)
test_df_D=test_df_D.fillna(0)
from imblearn.combine import SMOTETomek
# from imblearn.over_sampling import SMOTE
smt = SMOTETomek(ratio='auto')
df_D_over, df_Y_over = smt.fit_resample(df_D, df['Y'])
df_D_over = pd.DataFrame(df_D_over)
test_df_D.columns = df_D_over.columns

procs = [FillMissing, Categorify, Normalize]
num_cal = df_D_over.columns.values
cat_col = []

df_D_over['Y'] = df_Y_over

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [4]:
fDF = df_D_over
fTDF = test_df_D
data = (TabularList.from_df(fDF, procs=procs, cont_names=num_cal, cat_names=cat_col)
          .split_by_idx(valid_idx=range(int(len(df_D)*0.9),len(df_D)))
          .label_from_df(cols=dep_var)
          .add_test(TabularList.from_df(fTDF, cont_names=num_cal, cat_names=cat_col, procs=procs))
          .databunch())

learn = tabular_learner(data, layers=[50, 50, 20], metrics=accuracy)
learn.fit_one_cycle(40)

epoch,train_loss,valid_loss,accuracy,time
0,0.345874,0.358833,0.853191,00:06
1,0.288224,0.283864,0.880496,00:05
2,0.240106,0.275736,0.876950,00:05
3,0.223128,0.240264,0.895390,00:05
4,0.215219,0.274866,0.875886,00:05
5,0.210744,0.239918,0.895390,00:05
6,0.204603,0.226168,0.898582,00:05
7,0.190491,0.232473,0.896099,00:05
8,0.184601,0.228350,0.897518,00:05
9,0.186157,0.219161,0.903901,00:05


In [30]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

le_y = le.inverse_transform(pred_class)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission002.csv', index=False)
from google.colab import files
files.download('submission.csv')